In [1]:
!mkdir ~/.kaggle

In [2]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets list -s 'Leaf Disease Dataset (combination)'


ref                                                         title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
asheniranga/leaf-disease-dataset-combination                Leaf Disease Dataset (combination)           761MB  2022-01-09 14:23:28           2451         45  0.75             
alinedobrovsky/plant-disease-classification-merged-dataset  Plant Disease Classification Merged Dataset   17GB  2023-01-03 15:11:19           5909         68  0.6875           
tolgahayit/yellowrust19-yellow-rust-disease-in-wheat        YELLOW-RUST-19                                 1GB  2021-11-02 13:33:09            656         10  0.5625           
amandam1/healthy-vs-diseased-leaf-image-dataset             Healthy vs. Diseased Leaf Image Dataset        6GB  202

In [4]:
!kaggle datasets download -d asheniranga/leaf-disease-dataset-combination

Dataset URL: https://www.kaggle.com/datasets/asheniranga/leaf-disease-dataset-combination
License(s): GPL-2.0
100% 759M/761M [00:06<00:00, 155MB/s]
100% 761M/761M [00:06<00:00, 116MB/s]


In [5]:
! unzip leaf-disease-dataset-combination.zip

Streaming output truncated to the last 5000 lines.
  inflating: image data/validation/peach/bacterial spot/7e74be0c-cd8c-45fe-80db-66bb30a8a4cc___Rut._Bact.S 0826.JPG  
  inflating: image data/validation/peach/bacterial spot/7f481c3f-bc25-470f-a47b-64962897dae0___Rutg._Bact.S 2246.JPG  
  inflating: image data/validation/peach/bacterial spot/7fd55331-be6b-4554-bdea-009dda8a1302___Rutg._Bact.S 2183.JPG  
  inflating: image data/validation/peach/bacterial spot/81020371-a3d7-421e-94fc-f44cf46be444___Rut._Bact.S 1337.JPG  
  inflating: image data/validation/peach/bacterial spot/810a3211-f2d6-4e0e-86e6-b241946a5c69___Rutg._Bact.S 1206.JPG  
  inflating: image data/validation/peach/bacterial spot/81b31538-0772-47f0-bb9d-3799fa2d296d___Rutg._Bact.S 1549.JPG  
  inflating: image data/validation/peach/bacterial spot/81d21a0e-8551-41e5-b8d6-b5840fc6cda9___Rut._Bact.S 1400.JPG  
  inflating: image data/validation/peach/bacterial spot/82bd1f8e-ec90-4f92-9d62-6f40ebe3f4f3___Rut._Bact.S 0941.JPG  
 

In [6]:
!mv 'image data' image_data

In [7]:
%cd image_data

/content/image_data


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
#For working with images:
import PIL
import cv2
import matplotlib.image as mpimg
from PIL import Image
import shutil
from PIL import Image, ImageFilter
import seaborn as sns
from PIL import Image, ImageEnhance
from PIL import Image, ImageFilter, ImageOps

In [19]:
 # Define the base path
base_path = '/content/image_data'  # Updated path
# Define the subdirectories
subdirs = ['test', 'train', 'validation']

In [10]:
# Function to check if an image is blurry
def is_blurry(image_path, threshold=100):
  image = cv2.imread(image_path)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  fm = cv2.Laplacian(gray, cv2.CV_64F).var()
  if fm < threshold:
    return True
  else:
    return False


# Iterate through each subdirectory
for subdir in subdirs:
    tomato_path = os.path.join(base_path, subdir, 'tomato')
    if os.path.exists(tomato_path):
        print(f"Contents of {tomato_path}:")
        classes = [item for item in os.listdir(tomato_path) if os.path.isdir(os.path.join(tomato_path, item))]
        for class_name in classes:
            class_path = os.path.join(tomato_path, class_name)
            images = [img for img in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, img))]
            blurry_images = []
            for img_name in images:
                img_path = os.path.join(class_path, img_name)
                if is_blurry(img_path):
                    blurry_images.append(img_name)
            print(f"  - {class_name}: {len(blurry_images)} blurry images")
    else:
        print(f"{tomato_path} does not exist.")

Contents of /content/image_data/test/tomato:
  - tomato yellow leaf curl virus: 113 blurry images
  - early blight: 0 blurry images
  - healthy: 0 blurry images
  - late blight: 2 blurry images
  - tomato mosaic virus: 0 blurry images
  - bacterial spot: 1 blurry images
  - septoria leaf spot: 0 blurry images
  - leaf mold: 0 blurry images
  - spider mites two-spotted spider mite: 0 blurry images
  - target spot: 0 blurry images
Contents of /content/image_data/train/tomato:
  - tomato yellow leaf curl virus: 932 blurry images
  - early blight: 2 blurry images
  - healthy: 0 blurry images
  - late blight: 17 blurry images
  - tomato mosaic virus: 0 blurry images
  - bacterial spot: 2 blurry images
  - septoria leaf spot: 0 blurry images
  - leaf mold: 0 blurry images
  - spider mites two-spotted spider mite: 0 blurry images
  - target spot: 0 blurry images
Contents of /content/image_data/validation/tomato:
  - tomato yellow leaf curl virus: 219 blurry images
  - early blight: 0 blurry i

## **Data preprocessing**

### **Handling the blurry images**


In [11]:
# Function to sharpen an image using unsharp masking
def sharpen_image(image_path, radius=2, amount=1.0, threshold=0):
  """
  Sharpens an image using unsharp masking.

  Args:
    image_path: Path to the image file.
    radius: Blur radius for the Gaussian blur.
    amount: Strength of the sharpening effect.
    threshold: Threshold for the unsharp mask.

  Returns:
    The sharpened image.
  """
  try:
    image = Image.open(image_path)
    blurred_image = image.filter(ImageFilter.GaussianBlur(radius=radius))
    sharpened_image = Image.blend(image, blurred_image, alpha=-amount)
    return sharpened_image
  except Exception as e:
    print(f"Error processing image {image_path}: {e}")
    return None

# Iterate through each subdirectory
for subdir in subdirs:
    tomato_path = os.path.join(base_path, subdir, 'tomato')
    if os.path.exists(tomato_path):
        print(f"Contents of {tomato_path}:")
        classes = [item for item in os.listdir(tomato_path) if os.path.isdir(os.path.join(tomato_path, item))]
        for class_name in classes:
            class_path = os.path.join(tomato_path, class_name)
            images = [img for img in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, img))]
            for img_name in images:
                img_path = os.path.join(class_path, img_name)
                if is_blurry(img_path):
                    sharpened_img = sharpen_image(img_path)
                    if sharpened_img:
                        plt.imshow(sharpened_img)
                        plt.title(f"Sharpened Image: {img_name}")
                        plt.show()
    else:
        print(f"{tomato_path} does not exist.")

Output hidden; open in https://colab.research.google.com to view.